<a href="https://colab.research.google.com/github/Aleezahshaikh/Bioinformatics_problems/blob/main/GLOB/Global_Alignment_with_Scoring_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 15.2 MB/s eta 0:00:00


In [6]:
import numpy as np
from Bio import SeqIO
from google.colab import files

GAP_PENALTY = -5

BLOSUM62 = {
    ('A', 'A'): 4, ('A', 'C'): 0, ('A', 'D'): -2, ('A', 'E'): -1, ('A', 'F'): -2, ('A', 'G'): 0, ('A', 'H'): -2,
    ('A', 'I'): -1, ('A', 'K'): -1, ('A', 'L'): -1, ('A', 'M'): -1, ('A', 'N'): -2, ('A', 'P'): -1, ('A', 'Q'): -1,
    ('A', 'R'): -1, ('A', 'S'): 1, ('A', 'T'): 0, ('A', 'V'): 0, ('A', 'W'): -3, ('A', 'Y'): -2,
    ('C', 'C'): 9, ('C', 'D'): -3, ('C', 'E'): -4, ('C', 'F'): -2, ('C', 'G'): -3, ('C', 'H'): -3, ('C', 'I'): -1,
    ('C', 'K'): -3, ('C', 'L'): -1, ('C', 'M'): -1, ('C', 'N'): -3, ('C', 'P'): -3, ('C', 'Q'): -3, ('C', 'R'): -3,
    ('C', 'S'): -1, ('C', 'T'): -1, ('C', 'V'): -1, ('C', 'W'): -2, ('C', 'Y'): -2,
    ('D', 'D'): 6, ('D', 'E'): 2, ('D', 'F'): -3, ('D', 'G'): -1, ('D', 'H'): -1, ('D', 'I'): -3, ('D', 'K'): -1,
    ('D', 'L'): -4, ('D', 'M'): -3, ('D', 'N'): 1, ('D', 'P'): -1, ('D', 'Q'): 0, ('D', 'R'): -2, ('D', 'S'): 0,
    ('D', 'T'): -1, ('D', 'V'): -3, ('D', 'W'): -4, ('D', 'Y'): -3,
    ('E', 'E'): 5, ('E', 'F'): -3, ('E', 'G'): -2, ('E', 'H'): 0, ('E', 'I'): -3, ('E', 'K'): 1, ('E', 'L'): -3,
    ('E', 'M'): -2, ('E', 'N'): 0, ('E', 'P'): -1, ('E', 'Q'): 2, ('E', 'R'): 0, ('E', 'S'): 0, ('E', 'T'): -1,
    ('E', 'V'): -2, ('E', 'W'): -3, ('E', 'Y'): -2,
    ('F', 'F'): 6, ('F', 'G'): -3, ('F', 'H'): -1, ('F', 'I'): 0, ('F', 'K'): -3, ('F', 'L'): 0, ('F', 'M'): 0,
    ('F', 'N'): -3, ('F', 'P'): -4, ('F', 'Q'): -3, ('F', 'R'): -3, ('F', 'S'): -2, ('F', 'T'): -2, ('F', 'V'): -1,
    ('F', 'W'): 1, ('F', 'Y'): 3,
    ('G', 'G'): 6, ('G', 'H'): -2, ('G', 'I'): -4, ('G', 'K'): -2, ('G', 'L'): -4, ('G', 'M'): -3, ('G', 'N'): 0,
    ('G', 'P'): -2, ('G', 'Q'): -2, ('G', 'R'): -2, ('G', 'S'): 0, ('G', 'T'): -2, ('G', 'V'): -3, ('G', 'W'): -2,
    ('G', 'Y'): -3,
    ('H', 'H'): 8, ('H', 'I'): -3, ('H', 'K'): -1, ('H', 'L'): -3, ('H', 'M'): -2, ('H', 'N'): 1, ('H', 'P'): -2,
    ('H', 'Q'): 0, ('H', 'R'): 0, ('H', 'S'): -1, ('H', 'T'): -2, ('H', 'V'): -3, ('H', 'W'): -2, ('H', 'Y'): 2,
    ('I', 'I'): 4, ('I', 'K'): -3, ('I', 'L'): 2, ('I', 'M'): 1, ('I', 'N'): -3, ('I', 'P'): -3, ('I', 'Q'): -3,
    ('I', 'R'): -3, ('I', 'S'): -2, ('I', 'T'): -1, ('I', 'V'): 3, ('I', 'W'): -3, ('I', 'Y'): -1,
    ('K', 'K'): 5, ('K', 'L'): -2, ('K', 'M'): -1, ('K', 'N'): 0, ('K', 'P'): -1, ('K', 'Q'): 1, ('K', 'R'): 2,
    ('K', 'S'): 0, ('K', 'T'): -1, ('K', 'V'): -2, ('K', 'W'): -3, ('K', 'Y'): -2,
    ('L', 'L'): 4, ('L', 'M'): 2, ('L', 'N'): -3, ('L', 'P'): -3, ('L', 'Q'): -2, ('L', 'R'): -2, ('L', 'S'): -2,
    ('L', 'T'): -1, ('L', 'V'): 1, ('L', 'W'): -2, ('L', 'Y'): -1,
    ('M', 'M'): 5, ('M', 'N'): -2, ('M', 'P'): -2, ('M', 'Q'): 0, ('M', 'R'): -1, ('M', 'S'): -1, ('M', 'T'): -1,
    ('M', 'V'): 1, ('M', 'W'): -1, ('M', 'Y'): -1,
    ('N', 'N'): 6, ('N', 'P'): -2, ('N', 'Q'): 0, ('N', 'R'): 0, ('N', 'S'): 1, ('N', 'T'): 0, ('N', 'V'): -3,
    ('N', 'W'): -4, ('N', 'Y'): -2,
    ('P', 'P'): 7, ('P', 'Q'): -1, ('P', 'R'): -2, ('P', 'S'): -1, ('P', 'T'): -1, ('P', 'V'): -2, ('P', 'W'): -4,
    ('P', 'Y'): -3,
    ('Q', 'Q'): 5, ('Q', 'R'): 1, ('Q', 'S'): 0, ('Q', 'T'): -1, ('Q', 'V'): -2, ('Q', 'W'): -2, ('Q', 'Y'): -1,
    ('R', 'R'): 5, ('R', 'S'): -1, ('R', 'T'): -1, ('R', 'V'): -3, ('R', 'W'): -3, ('R', 'Y'): -2,
    ('S', 'S'): 4, ('S', 'T'): 1, ('S', 'V'): -2, ('S', 'W'): -3, ('S', 'Y'): -2,
    ('T', 'T'): 5, ('T', 'V'): 0, ('T', 'W'): -2, ('T', 'Y'): -2,
    ('V', 'V'): 4, ('V', 'W'): -3, ('V', 'Y'): -1,
    ('W', 'W'): 11, ('W', 'Y'): 2,
    ('Y', 'Y'): 7
}

def blosum62_score(a, b):
    return BLOSUM62.get((a, b), BLOSUM62.get((b, a), -1))

def get_protein_sequences():
    """
    Prompts the user to input protein sequences manually or upload a FASTA file.
    Validates the input based on limits for the number (exactly 2) and length (up to 1000 aa) of sequences.

    Returns:
        list: A list of two protein sequences if valid; otherwise, an empty list.
    """
    choice = input("CHOOSE \n (1) Do you want to input a sequence manually or \n (2) Upload a FASTA file \nEnter 1 or 2: ")

    if choice == '1':
        sequences = []
        for i in range(2):
            seq = input(f"Enter protein sequence {i + 1} (up to 1000 aa): ").strip().upper()
            if len(seq) > 1000:
                print("Error: Sequence length exceeds 1000 amino acids.")
                return []
            sequences.append(seq)
        return sequences
    elif choice == '2':
        print("Please upload a FASTA file.")
        uploaded = files.upload()
        uploaded_file = list(uploaded.keys())[0]
        sequences = [str(record.seq) for record in SeqIO.parse(uploaded_file, "fasta")]
        if len(sequences) != 2:
            print("Error: FASTA file must contain exactly two sequences.")
            return []
        if any(len(seq) > 1000 for seq in sequences):
            print("Error: One or more sequences in the file exceed 1000 amino acids.")
            return []
        return sequences
    else:
        print("Invalid choice. Please enter 1 for manual input or 2 for file upload.")
        return get_protein_sequences()

def needleman_wunsch(seq1, seq2):
    n = len(seq1)
    m = len(seq2)
    dp = np.zeros((n + 1, m + 1), dtype=int)
    for i in range(1, n + 1):
        dp[i][0] = dp[i - 1][0] + GAP_PENALTY
    for j in range(1, m + 1):
        dp[0][j] = dp[0][j - 1] + GAP_PENALTY
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = dp[i - 1][j - 1] + blosum62_score(seq1[i - 1], seq2[j - 1])
            delete = dp[i - 1][j] + GAP_PENALTY
            insert = dp[i][j - 1] + GAP_PENALTY
            dp[i][j] = max(match, delete, insert)
    return dp[n][m]

sequences = get_protein_sequences()

if sequences:
    seq1, seq2 = sequences
    alignment_score = needleman_wunsch(seq1, seq2)
    print("Maximum Alignment Score:", alignment_score)


#expected output for the following input sample is 35
# >sequence1
# MEEPQSDPSV
# >sequence2
# MEESQDPSV

CHOOSE 
 (1) Do you want to input a sequence manually or 
 (2) Upload a FASTA file 
Enter 1 or 2: 1
Enter protein sequence 1 (up to 1000 aa): MEEPQSDPSV
Enter protein sequence 2 (up to 1000 aa): MEESQDPSV
Maximum Alignment Score: 35
